In [1]:
import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
import pandas as pd 
import re 
from nltk.corpus import stopwords

In [2]:
df = pd.read_csv('/Users/Moon/nlp-tutorial/tripadvisor_hotel_reviews.csv')
df = df.sample(n = 2000)

In [3]:
print(df.isnull().sum())

Review    0
Rating    0
dtype: int64


In [4]:
def categorical(data):
    a = {i+1: w for i,w in enumerate(range(len(data.unique())))}
    b = []
    for i in df['Rating']:
        one_hot = [0]*(len(df['Rating'].unique()))
        index = a[i]
        one_hot[index] = 1
        b.append(one_hot)
    return b




In [5]:
label = categorical(df['Rating'])

In [6]:
def preprocessing(data):
    only = []
    for i in data:
        only.append(re.sub('[^a-zA-Z]', ' ', i))
    
    stops = set(stopwords.words('english'))
    inputs = [word for word in only if not word in stops]
    
    return inputs

In [7]:
data = preprocessing(df['Review'])

In [8]:
b = []
for i in data:
    b.append(len(i))

In [9]:
import seaborn as sns
sns.distplot(b)

In [10]:
b = pd.Series(b)
b.describe()

count     2000.000000
mean       730.469000
std        670.060264
min         60.000000
25%        351.000000
50%        540.000000
75%        875.250000
max      10062.000000
dtype: float64

In [11]:
c = pd.Series(b).describe()

In [21]:
g = c['75%']

In [22]:
g

875.25

In [23]:
x = [i for i in data]
word_list = ' '.join(x).split()
word_list = list(set(word_list))

word_dict = {w : i for i, w in enumerate(word_list)}
vocab_size = len(word_dict)


In [24]:
X = []
for sen in data:
        word = sen.split()
        
        input_x = [word_dict[n] for n in word]
        X.append(input_x[:int(g)])

In [27]:
int(g)

875

In [29]:
len(X[0])

82

In [33]:
max_len = int(g)

def batch():
    X = []
    y = []
    
    for sen in data:
        word = sen.split()
        
        input_x = [word_dict[n] for n in word]
        X.append(input_x[:max_len])
        
    for item in X:
        while len(item) < max_len:
            item.append(0)
            
    y = [i for i in categorical(df['Rating'])]

    
    return X, y



In [34]:
X,y = batch()

In [39]:
X = torch.LongTensor(X)
y = torch.FloatTensor(y)

In [40]:
print(y.shape)

torch.Size([2000, 5])


In [56]:
hidden = 3
embedding_size = 5
C = nn.Embedding(vocab_size, embedding_size)
lstm = nn.LSTM(input_size = embedding_size, hidden_size = hidden)
W = nn.Linear(max_len * hidden, 5)

In [57]:
vocab_size

15619

In [58]:
X.shape

torch.Size([2000, 875])

In [59]:
C(X).shape

torch.Size([2000, 875, 5])

In [60]:
embed = C(X)
print(embed.shape)
print(embed)

torch.Size([2000, 875, 5])
tensor([[[-0.1026, -1.7125, -1.0141, -1.8509, -0.3761],
         [ 0.2630,  0.2868, -1.0521, -0.8570,  1.5943],
         [-0.4577, -0.8166, -0.5232,  0.8071, -0.4960],
         ...,
         [ 0.1232,  1.2420,  1.6033,  0.7651,  0.9256],
         [ 0.1232,  1.2420,  1.6033,  0.7651,  0.9256],
         [ 0.1232,  1.2420,  1.6033,  0.7651,  0.9256]],

        [[-2.6931, -0.6384,  0.8434,  1.2591,  0.0625],
         [ 0.4339,  1.3177, -0.2733, -1.2634, -2.4958],
         [ 0.4282, -0.1981, -1.5471,  0.6868, -2.2018],
         ...,
         [ 0.1232,  1.2420,  1.6033,  0.7651,  0.9256],
         [ 0.1232,  1.2420,  1.6033,  0.7651,  0.9256],
         [ 0.1232,  1.2420,  1.6033,  0.7651,  0.9256]],

        [[ 0.3445,  0.8286,  0.4700,  1.3043,  0.3588],
         [-1.1043, -1.1245,  0.8650,  0.1754,  1.0857],
         [ 0.2116,  0.0961,  0.6255, -0.5220, -0.4737],
         ...,
         [ 0.1232,  1.2420,  1.6033,  0.7651,  0.9256],
         [ 0.1232,  1.2420,  1.

In [61]:
print(embed.shape)
print(embed)

torch.Size([2000, 875, 5])
tensor([[[-0.1026, -1.7125, -1.0141, -1.8509, -0.3761],
         [ 0.2630,  0.2868, -1.0521, -0.8570,  1.5943],
         [-0.4577, -0.8166, -0.5232,  0.8071, -0.4960],
         ...,
         [ 0.1232,  1.2420,  1.6033,  0.7651,  0.9256],
         [ 0.1232,  1.2420,  1.6033,  0.7651,  0.9256],
         [ 0.1232,  1.2420,  1.6033,  0.7651,  0.9256]],

        [[-2.6931, -0.6384,  0.8434,  1.2591,  0.0625],
         [ 0.4339,  1.3177, -0.2733, -1.2634, -2.4958],
         [ 0.4282, -0.1981, -1.5471,  0.6868, -2.2018],
         ...,
         [ 0.1232,  1.2420,  1.6033,  0.7651,  0.9256],
         [ 0.1232,  1.2420,  1.6033,  0.7651,  0.9256],
         [ 0.1232,  1.2420,  1.6033,  0.7651,  0.9256]],

        [[ 0.3445,  0.8286,  0.4700,  1.3043,  0.3588],
         [-1.1043, -1.1245,  0.8650,  0.1754,  1.0857],
         [ 0.2116,  0.0961,  0.6255, -0.5220, -0.4737],
         ...,
         [ 0.1232,  1.2420,  1.6033,  0.7651,  0.9256],
         [ 0.1232,  1.2420,  1.

In [62]:
o, (_,_) = lstm(embed)

In [63]:
print(o.shape)
print(o)

torch.Size([2000, 875, 3])
tensor([[[-0.3453, -0.3764, -0.0859],
         [-0.3925, -0.0280, -0.0440],
         [-0.1480, -0.0808, -0.0664],
         ...,
         [ 0.1968,  0.3957, -0.0873],
         [ 0.1968,  0.3957, -0.0873],
         [ 0.1968,  0.3957, -0.0873]],

        [[-0.1910, -0.1305, -0.0585],
         [ 0.0687, -0.1941, -0.3602],
         [-0.0231, -0.1651, -0.1972],
         ...,
         [ 0.2436,  0.5364, -0.1361],
         [ 0.2436,  0.5364, -0.1361],
         [ 0.2436,  0.5364, -0.1361]],

        [[ 0.0501,  0.2708, -0.1472],
         [-0.2625,  0.1637, -0.0775],
         [ 0.0105, -0.2994, -0.2958],
         ...,
         [ 0.2574,  0.5904, -0.1577],
         [ 0.2574,  0.5904, -0.1577],
         [ 0.2574,  0.5904, -0.1577]],

        ...,

        [[-0.4665, -0.3947, -0.1442],
         [-0.4254,  0.0876, -0.2616],
         [-0.3482,  0.4923, -0.1288],
         ...,
         [ 0.2629,  0.6350, -0.1768],
         [ 0.2629,  0.6350, -0.1768],
         [ 0.2629,  0.6

In [64]:
c = o.view(-1, max_len*3)
c.shape

torch.Size([2000, 2625])

In [66]:
a = o.view(-1, int(g)*hidden)
print(a.shape)

torch.Size([2000, 2625])


In [67]:
y_hat = W(a)
print(y_hat.shape)
print(y.shape)
y_hat = y_hat.squeeze()

torch.Size([2000, 5])
torch.Size([2000, 5])


In [69]:
criterion = nn.CrossEntropyLoss()


In [73]:
print(y.shape)
print(y_hat.shape)

torch.Size([2000, 5])
torch.Size([2000, 5])


In [76]:
a = y.index
print(a)

AttributeError: 'Tensor' object has no attribute 'index'

In [75]:
y_hat

tensor([[ 0.0885,  0.2471,  0.1159, -0.1826,  0.1813],
        [ 0.0801,  0.3519,  0.2286, -0.2638,  0.2386],
        [ 0.0726,  0.4127,  0.2536, -0.3618,  0.3481],
        ...,
        [ 0.1208,  0.4462,  0.2889, -0.3228,  0.2618],
        [ 0.2538,  0.5439,  0.2563, -0.4772,  0.2297],
        [ 0.2143,  0.4393,  0.3474, -0.3684,  0.2787]],
       grad_fn=<SqueezeBackward0>)

In [78]:
c = nn.MultiLabelSoftMarginLoss()
loss = c(y_hat, y)
print(loss)

tensor(0.7741, grad_fn=<MeanBackward0>)
